In [1]:
import functools

import regex as re
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from transformers.pipelines.pt_utils import KeyDataset

In [43]:
train_df = pd.read_csv('./data/train.csv')
eval_df = pd.read_csv('./data/eval-Mistral.csv')

In [44]:
eval_ds = Dataset.from_pandas(eval_df[['joke', 'explanation']])

d:\Program\miniconda3\envs\DL\lib\site-packages\pyarrow\pandas_compat.py:358: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    trust_remote_code=True,
    use_safetensors=True,
)
quan_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type='nf4',
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map='auto',
    trust_remote_code=True,
    use_safetensors=True,
    torch_dtype=torch.bfloat16,
    quantization_config=quan_config,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
# zero-shot
prompt = """<s>[INST]You are a joke explainer.

Each input joke contains a question and an answer. Your job is to explain why the answer together with the question forms a joke.
Use no more than 3-4 sentences to explain the joke. Do not add "Ah I see" or similar terms to your explanation.

%s[/INST]"""

In [38]:
# one-shot
prompt = """<s>[INST]You are a joke explainer.

Each input joke contains a question and an answer. Your job is to explain why the answer together with the question forms a joke.

Here are some examples of joke explanations:
Joke: Q: Why didn't the string ever win a race? A: It always tied!
Explanation: The joke plays on the double meaning of the word 'tied.' In one sense, 'tied' means to finish a race at the same time as another competitor, resulting in a draw. In another sense, 'tied' refers to the action of tying a knot. Since the subject of the joke is a 'string,' the humor arises from imagining the string as always tying itself into knots, which humorously explains why it could never win a race.

Use no more than 3-4 sentences to explain the joke. Do not add "Ah I see" or similar terms to your explanation.

%s[/INST]"""

In [45]:
# five-shot
prompt = """<s>[INST]You are a joke explainer.

Each input joke contains a question and an answer. Your job is to explain why the answer together with the question forms a joke.

Here are some examples of joke explanations:
1.
Joke: Q: Why didn't the string ever win a race? A: It always tied!
Explanation: The joke plays on the double meaning of the word 'tied.' In one sense, 'tied' means to finish a race at the same time as another competitor, resulting in a draw. In another sense, 'tied' refers to the action of tying a knot. Since the subject of the joke is a 'string,' the humor arises from imagining the string as always tying itself into knots, which humorously explains why it could never win a race.
2.
Joke: Q: Why did the golfer wear two pairs of pants? A: In case he got a hole in one!
Explanation: This joke plays on the double meaning of the phrase 'hole in one'. In golf, 'a hole in one' refers to a player hitting the ball directly from the tee into the hole with one stroke, which is a very good outcome. However, in the context of wearing pants, 'a hole in one' humorously suggests a literal hole in one of the pairs of pants. Thus, the golfer wearing two pairs of pants is a silly precaution to ensure he still has one intact pair if one gets a hole.
3.
Joke: Q:What do prisoners use to call each other? A:Cell phones!
Explanation: The joke plays on the double meaning of the word 'cell.' In one sense, a 'cell' is a small room where prisoners are kept, and in another sense, it refers to 'cellular phones,' commonly known as cell phones. The humor arises from the pun created by using 'cell phones' to imply that prisoners would use phones related to their incarceration cells to communicate.
4.
Joke: Q: What kind of shoes does a ninja wear? A: Sneakers!
Explanation: The joke plays on the word 'sneakers' which is a type of shoe known for being comfortable and quiet, ideal for casual wear. However, in the context of the joke, 'sneakers' also refers to the ability to sneak around silently, which is a stereotypical attribute of a ninja. Thus, the humor arises from the double meaning of 'sneakers' as both a type of shoe and a description of a ninja's stealthy behavior.
5.
Joke: Q: Why did Cinderella get kicked off the soccer team? A: Because she ran away from the ball!
Explanation: The joke plays on the double meaning of the word 'ball'. In the context of the story of Cinderella, 'ball' refers to the formal dance she attends, from which she famously runs away at midnight. In the context of soccer, 'ball' refers to the soccer ball, which players should not run away from if they want to play effectively. The humor arises from the absurdity of applying the fairy tale behavior to the soccer field, creating a funny and unexpected reason for Cinderella to be kicked off the team.

Use no more than 3-4 sentences to explain the joke. Do not add "Ah I see" or similar terms to your explanation.

%s[/INST]"""

In [46]:
eval_ds = eval_ds.map(lambda x: {'prompt': prompt % x['joke']})

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [9]:
assist_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [47]:
results = assist_pipeline(KeyDataset(eval_ds, 'prompt'),
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                          max_length=1000,
                          return_full_text=False)

In [48]:
ans_ls = []
for res in results:
    ans_ls.append(re.sub(r'\s+', ' ', res[0]['generated_text']).strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [49]:
eval_df['five-shot'] = ans_ls
eval_df.to_csv('./data/eval-Mistral.csv', index=False)